In [ ]:
# import libraries
import numpy as np
import sys
import psi4
from helper_PFCI import PFHamiltonianGenerator
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', False)
import time
import json

We are going to read the energy eigenvalues and dipole matrix elements from .npy files.  We will also still create an instance of the PFHamiltonianGenerator class so we can use its build_pcqed_pf_hamiltonian() method, but it is not really important what details we use to instantiate this class... so we will use LiH in a minimal basis since this is a fast way to instantiate the class!

In [ ]:
# read data from .npy files for formaldehyde casci(8,8) calculations

# !!! Change this to the correct path on your computer!
npy_folder = "/Users/rmandern/code/SCQED-PCQED/Formaldehyde/"

# these file names should still be good
E_npy_file = npy_folder + "CH2O_ccpVDZ_CASCI_88_1000_dipole_allowed_E_Array.npy"
Mu_npy_file = npy_folder + "CH2O_ccpVDZ_CASCI_88_1000_dipole_allowed_Mu_Array.npy"

# store energy eigenvalues in E_array
E_array = np.load(E_npy_file)
# store dipole matrix elements in Mu_array
Mu_array = np.load(Mu_npy_file)

# print their shape so we know how many elements we have
print(np.shape(E_array))
print(np.shape(Mu_array))

# setup basic arguments to create an instance of the PFHamiltonianGenerator class
mol_str = """
Li
H 1 1.8
symmetry c1
"""

options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}


cavity_free_dict = {
    'omega_value' : 0.0,
    'lambda_vector' : np.array([0, 0, 0.0]),
    'ci_level' : 'fci',   
    'full_diagonalization' : True,
    'number_of_photons' : 0, 
}

# create the instance of our PFHamiltonianGenerator class
instance = PFHamiltonianGenerator(mol_str, options_dict, cavity_free_dict)

In [ ]:
# open json file
f = open("CH2O_CO_r_1.3_casci_cc-pVDZ_n_act_els_8_n_act_orbs_8_dipole_allowed.json")
data = json.load(f)

In [ ]:
# get the array of lambda values from the json file
lambdas = data["model"]["lambda_vector"]

# determine how many lambda values will be in the scan
N_l = len(lambdas)
print(N_l)

In [ ]:
fast_start = time.time()
# instance.fast_build_pcqed_pf_hamiltonian(N_el, N_ph, omega, lambda_vector, E_array, Mu_array)
fast_end = time.time()
print(F"Fast build took {fast_end-fast_start} seconds")

In [ ]:
# add lambda values from jason file 
# cal energy by looping and save them agian in jason file 



N_el = 5
N_ph = 2
omega = 0.12086
lambda_vector = np.array([0., 0., 0.05])

fast_start = time.time()
# create an array of zeros to store the PCQED eigenvalues for each value of lambda
# _pcqed_52 = np.zeros((N_l, N_el * N_ph))
# loop over values of lambda, build Hamiltonian, capture eigenvalues
ctr = 0
for lam in lambdas:
    _HPF = instance.fast_build_pcqed_pf_hamiltonian(N_el, N_ph, omega, lambda_vector, E_array, Mu_array)
#     pf_e, pf_c = np.linalg.eigh(_HPF)
#     _pcqed_52[ctr, :] = pf_e
    ctr += 1
# instance.PCQED_HF_PF
E_pcqed_52 = instance.PCQED_eigs
# instance.PCQED_vecs
print(E_pcqed_52)
fast_end = time.time()
print(F"Fast build took {fast_end-fast_start} seconds")

In [ ]:
N_el = 200
N_ph = 2
omega = 0.12086
lambda_vector = np.array([0., 0., 0.05])

fast_start = time.time()
# create an array of zeros to store the PCQED eigenvalues for each value of lambda
# _pcqed_52 = np.zeros((N_l, N_el * N_ph))
# loop over values of lambda, build Hamiltonian, capture eigenvalues
ctr = 0
for lam in lambdas:
    _HPF = instance.fast_build_pcqed_pf_hamiltonian(N_el, N_ph, omega, lambda_vector, E_array, Mu_array)
#     pf_e, pf_c = np.linalg.eigh(_HPF)
#     _pcqed_52[ctr, :] = pf_e
    ctr += 1
# instance.PCQED_HF_PF
E_pcqed_2002 = instance.PCQED_eigs
# instance.PCQED_vecs
print(E_pcqed_2002)
fast_end = time.time()
print(F"Fast build took {fast_end-fast_start} seconds")

In [ ]:
data_2002 = data
#print(data_52["return_result"])
data_2002["model"]["number_of_electronic_states"] = 200
data_2002["model"]["number_of_photonic_states"] = 2

# loop over the different lambda values
energies = []
for i in range(len(E_pcqed_2002[:,0])):
    sub_energies = []
    for j in range(len(E_pcqed_2002[0,:])):
        sub_energies.append(E_pcqed_2002[i,j])
    energies.append(sub_energies)

print(energies)

data_2002["return_result"] = energies

file_name = "CH2O_CO_r_1.3_pcqed_2002.json"
json_object = json.dumps(data_2002, indent=4)
with open(file_name, "w") as outfile:
    outfile.write(json_object)

In [ ]:
N_el = 500
N_ph = 2
omega = 0.12086
# lambda_vector = np.array([0., 0., 0.05])

fast_start = time.time()
# create an array of zeros to store the PCQED eigenvalues for each value of lambda
# _pcqed_52 = np.zeros((N_l, N_el * N_ph))
# loop over values of lambda, build Hamiltonian, capture eigenvalues
ctr = 0
for lam in lambdas:
    _HPF = instance.fast_build_pcqed_pf_hamiltonian(N_el, N_ph, omega, lambda_vector, E_array, Mu_array)
#     pf_e, pf_c = np.linalg.eigh(_HPF)
#     _pcqed_52[ctr, :] = pf_e
    ctr += 1
# instance.PCQED_HF_PF
E_pcqed_5002 = instance.PCQED_eigs
# instance.PCQED_vecs
print(E_pcqed_5002)
fast_end = time.time()
print(F"Fast build took {fast_end-fast_start} seconds")

In [ ]:
data_42 = data
#print(data_52["return_result"])
data_42["model"]["number_of_electronic_states"] = 4
data_42["model"]["number_of_photonic_states"] = 2

# loop over the different lambda values
energies = []
for i in range(len(_pcqed_42[:,0])):
    sub_energies = []
    for j in range(len(_pcqed_42[0,:])):
        sub_energies.append(_pcqed_42[i,j])
    energies.append(sub_energies)

print(energies)

data_42["return_result"] = energies

file_name = "CH2O_CO_r_1.3_pcqed_42.json"
json_object = json.dumps(data_42, indent=4)
with open(file_name, "w") as outfile:
    outfile.write(json_object)

# Notes on the matrix blocks

\begin{align}\label{EQN:projected_matrix}
{\bf \mathcal{H}} =
&\begin{bmatrix}
{\bf E} + {\bf D}   & -\sqrt{\frac{\omega}{2}} {\bf d}  & 0 & \dots & 0 & 0 \\
-\sqrt{\frac{\omega}{2}}{\bf d} & {\bf E} + {\bf D} + {\bf \Omega}   & -\sqrt{\omega} {\bf d}  & \dots & 0 & 0 \\
0   &  -\sqrt{\omega}{\bf d} & {\bf E} + {\bf D} + 2{\bf \Omega} & \dots & 0  & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
0      &      0 &   0    &  \dots     & {\bf E} + {\bf D} + (N-1){\bf \Omega}    & -\sqrt{\frac{N \omega}{2}} {\bf d} \\
0      &      0 &   0    &  \dots     & -\sqrt{\frac{N \omega}{2}}{\bf d}     & {\bf E} + {\bf D} + N{\bf \Omega}. 
\end{bmatrix}
\end{align}


Note that there are basically 4 different matrices with shape $(N_{el}, N_{el})$ that appear in this Hamiltonian matrix.  The following outlines the equations for the elements of each matrix along with one or more strategies to assemble them.  

### E matrix
#### Math
$$ E_{\alpha \beta} = \langle \psi_{\alpha} | \mathcal{H}_{el} | \psi_{\beta} \rangle = E_{\beta} \delta_{\alpha \beta} $$ 
where $E_{\beta}$ denote the electronic energy eigenvalues of the molecular system.
#### Code
If the energy eigenvalues are stored in an array called `E_array`, then we can build ${\bf E}$ by multiplying these values by an $(N_{el}, N_{el})$ identity matrix $\mathbb{I}$.

##### build N_el x N_el identity matrix
`_I = np.eye(N_el)`

##### build N_el x N_el _E matrix
`_E = E_array * _I`




In [ ]:
# code to build E matrix goes here
_I = np.eye(N_el)
_A = E_array[:N_el] * _I

### $\Omega$ matrix
#### Math
$$ \Omega_{\alpha \beta} = \langle \psi_{\alpha} | \omega | \psi_{\beta} \rangle  = \omega \delta_{\alpha \beta}$$ 

#### Code
If the photon frequency is stored in the variable `omega`, then the $(N_{el},N_{el})$ matrix `_O` can be build 
by multiplying the $\mathbb{I}$ by $omega$:

##### build N_el x N_el _O matrix
`_O = omega * _I`


In [ ]:
# code to build Omega matrix goes here
_O = omega * _I

### d matrix
#### Math
$$ d_{\alpha \beta} = \lambda \cdot  \langle \psi_{\alpha} | \mathcal{\mu} | \psi_{\beta} \rangle = \lambda_x \mathcal{\mu}_{x, \alpha \beta} +  \lambda_y \mathcal{\mu}_{y, \alpha \beta} +  \lambda_z \mathcal{\mu}_{z, \alpha \beta} $$ 
where $\mathcal{\mu}_{x, \alpha \beta}$ denotes the x-component of the (transition) dipole moment between molecular electronic state $\psi_{\alpha}$ and $\psi_{\beta}$.
#### Code
If the dipole matrix elements are stored in a $(N_{el}, N_{el}, 3)$  array called `mu_array` and the $\lambda$ vector is stored in an 3-element array called `\lambda_vector`, then we can build the
$(N_{el}, N_{el})$ array ${\bf d}$ by [contraction](https://en.wikipedia.org/wiki/Tensor_contraction) using `np.einsum()`:

##### build N_el x N_el _d matrix
`_d = np.einsum("k,ijk->ij", lambda_vector, mu_array)`

In [ ]:
# code to build the d matrix goes here
_d = np.einsum("k,ijk->ij", lambda_vector, Mu_array)

### D matrix
#### Math
$$ D_{\alpha \beta} = \frac{1}{2} \sum_{\gamma} d_{\alpha \gamma} d_{\gamma \beta} $$ 

#### Code
We have the $(N_{el}, N_{el})$ elements of the ${\bf d}$ array stored in `_d`.  We can then build `_D`
using matrix-matrix multiplication as follows:

`_D = _d @ _d`

or using einsum as follows:

`_D = np.einsum("ik,kj->ij",_d, _d)`


In [ ]:
# code to build the D matrix goes here
_D = _d @ _d
# _D = np.einsum("ik,kj->ij",_d, _d)